In [2]:
import torch as torch
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import os
import sys

sys.path.append('../')
import time
import bbobtorch
import gt_sampling



ModuleNotFoundError: No module named 'gt_sampling'

In [ ]:
import torch

def example_function(x):
    return x**4

def higher_order_derivative(f, wrt, n):
    if n == 0:
        return f
    else:
        grads = torch.autograd.grad(f, wrt, create_graph=True)[0]
        return higher_order_derivative(grads, wrt, n-1)

wrt = torch.tensor(3., requires_grad=True)
n = 3  # Change this to the desired order of derivative
result = higher_order_derivative(example_function(wrt), wrt, n)
print(result)


tensor(72., grad_fn=<MulBackward0>)


In [ ]:

# custom loss for derivatives of order n
def higher_order_derivatives(f, wrt, n):
    derivatives = [f.sum()]
    for f_ in f:    
        for _ in range(n):
            grads = torch.autograd.grad(f_, wrt, create_graph=True)[0]
            derivatives.append(grads.sum())
            f = grads.sum()
    
    return torch.stack(derivatives)

In [ ]:
class KnollHaZeHei(torch.nn.Module):
    def __init__(self, diff_degree, criterion):
        super().__init__()
        self.diff_degree = diff_degree
        self.criterion = criterion

    def forward(self, pred, true, x):
        pred = higher_order_derivatives(pred, x, self.diff_degree)
        true = higher_order_derivatives(true, x, self.diff_degree)
        return self.criterion(pred, true)


In [ ]:
import torch

def example_function(x):
    return 3*x**7-5*x**6



wrt = 1.
n = 3  # Change this to the desired order of derivative
wrt = torch.tensor([wrt, wrt], requires_grad=True)
f = example_function(wrt)
all_derivatives = higher_order_derivatives(f, wrt, n)
print(all_derivatives)


tensor([-4., -9., -9., -9., -9., -9., -9.], grad_fn=<StackBackward0>)


In [ ]:
# test
crit = KnollHaZeHei(2, torch.nn.MSELoss())
x = torch.tensor([3., 4], requires_grad=True)
target = x**2
pred = x**3

loss = crit(pred, target, x)

print(loss)

tensor(1687.6000, grad_fn=<MseLossBackward0>)


In [ ]:
# create ground truth functions
n_dim = 2
samples = 1000
seed = 42

def create_problem(f_number, n_dim, seed):
    if f_number == 1:
        problem = bbobtorch.create_f01(n_dim, seed=seed)
    elif f_number == 3:
        problem = bbobtorch.create_f03(n_dim, seed=seed)
    elif f_number == 24:
        problem = bbobtorch.create_f24(n_dim, seed=seed)

    return problem

problem_f01 = create_problem(1, n_dim, seed)
problem_f03 = create_problem(3, n_dim, seed)
problem_f24 = create_problem(24, n_dim, seed)

c:\Users\Pascal\DataScience\Anaconda\envs\MachineLearningProject\lib\site-packages\bbobtorch\utils.py:44: UserWarning: torch.qr is deprecated in favor of torch.linalg.qr and will be removed in a future PyTorch release.
The boolean parameter 'some' has been replaced with a string parameter 'mode'.
Q, R = torch.qr(A, some)
should be replaced with
Q, R = torch.linalg.qr(A, 'reduced' if some else 'complete') (Triggered internally at  ..\aten\src\ATen\native\BatchLinearAlgebra.cpp:2497.)
  R, _ = t.qr(R)


In [ ]:
# sampling from groundtruth functions
sample_f01 = gt_sampling.get_sample(problem_f01, n_samples=samples, n_dim=2, seed=42, method='random', )
# sample_f03 = gt_sampling.get_sample(problem_f03, n_samples=samples, n_dim=2, seed=42, method='random', requires_grad=True)
# sample_f24 = gt_sampling.get_sample(problem_f24, n_samples=samples, n_dim=2, seed=42, method='random', requires_grad=True)

NameError: name 'gt_sampling' is not defined